In [0]:
%sql
USE CATALOG bikestore;

In [0]:
# Definir pastas do projetos em variaveis para facilitar
bronze_path   = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/bronze/'
silver_path   = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/silver/'
gold_path     = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/gold/'
resource_path = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/resource/origem'
resource_path_volume = '/Volumes/bikestore/logistics/bikestore_rescource/origem/'

In [0]:
%python
silver_map = {
    "tmp_silver_customer":      f"{silver_path}/customer/",
    "tmp_silver_orders":        f"{silver_path}/orders_enriched",
    "tmp_silver_product":       f"{silver_path}/product/",

}
for view_name, path in silver_map.items():
    (spark.read.format('delta')
        .load(path)
        .createOrReplaceTempView(view_name))
 


In [0]:
%python
df_orders_pending = spark.sql("""
    WITH pending AS (
    SELECT 
        customer_id,
        order_date, 
        SUM(quantity) AS total_quantity,
        store_name,
        order_status_desc AS status
    FROM tmp_silver_orders
    WHERE lower(order_status_desc) = 'pending'
    GROUP BY customer_id, order_date, store_name, order_status_desc
    )
    SELECT 
    CONCAT(C.first_name, ' ', C.last_name) AS full_name,
    C.email,
    C.phone,
    P.* 
    FROM tmp_silver_customer AS C
    INNER JOIN pending AS P
    ON C.customer_id = P.customer_id
    WHERE C.email IS NOT NULL
    AND C.phone IS NOT NULL                              

""")

# salvar em Delta na gold
df_orders_pending.write\
    .mode('overwrite')\
    .format('delta')\
    .option('mergeSchema','true')\
    .save(f'{gold_path}/orders_pending')